# Data Visualization

The primary tools we'll use for plotting come from the [Holoviz](https://holoviz.org/) family of Python libraries, principally [GeoViews](https://geoviews.org/) and [hvPlot](https://hvplot.holoviz.org/). These are largely built on top of [HoloViews](https://holoviews.org/) and support multiple backends for rendering plots (notably [Bokeh](http://bokeh.pydata.org/) for interactive visualization and [Matplotlib](http://matplotlib.org/) for static, publication-quality plots).

## [GeoViews](https://geoviews.org/)

In [ ]:
import geoviews as gv
gv.extension('bokeh')
from geoviews import opts

### Displaying a basemap

- Principal utility is `gv.tile_sources`
- Use the method `opts` to specify optional settings
- Bokeh menu at right enables interactive exploration

In [ ]:
basemap = gv.tile_sources.OSM.opts(width=600, height=400)
basemap

# Plotting points

In [ ]:
tokyo_lonlat = (139.692222, 35.689722)
print(tokyo_lonlat)

+ `geoviews.Points` accepts a list of tuples (each of the form `(x, y)`) to plot.
+ Use the OpenStreetMap tiles from `gv.tile_sources.OSM` as a basemap.
+ Overlay using the Holoviews operator `*`
+ Define the options using `geoviews.opts`
+ ? find a way to initialize the zoom level sensibly?

In [ ]:
tokyo_point   = gv.Points([tokyo_lonlat])
point_opts = opts.Points(
                          size=48,
                          alpha=0.5,
                          color='red'
                        )

In [ ]:
# Use Holoviews * operator to overlay plot on basemap
# Note: zoom out to see basemap (starts zoomed "all the way in")
(basemap * tokyo_point).opts(point_opts)

In [ ]:
# to avoid starting zoomed all the way in, this zooms "all the way out"
(basemap * tokyo_point).opts(point_opts, opts.Overlay(global_extent=True))

## Plotting rectangles

+ Standard way to represent rectangle with corners
  $$(x_{\mathrm{min}},y_{\mathrm{min}}), (x_{\mathrm{min}},y_{\mathrm{max}}), (x_{\mathrm{max}},y_{\mathrm{min}}), (x_{\mathrm{max}},y_{\mathrm{max}})$$
  (assuming $x_{\mathrm{max}}>x_{\mathrm{min}}$ & $y_{\mathrm{max}}>y_{\mathrm{min}}$) is as a single 4-tuple
  $$(x_{\mathrm{min}},y_{\mathrm{min}},x_{\mathrm{max}},y_{\mathrm{max}}),$$
  i.e., the lower,left corner coordinates followed by the upper, right corner coordinates.

In [ ]:
# simple utility to make a rectangle of "half-width" dx & "half-height" dy & centred pt
def bounds(pt,dx,dy):
    '''Returns rectangle represented as tuple (x_lo, y_lo, x_hi, y_hi)
    given inputs pt=(x, y), half-width & half-height dx & dy respectively,
    where x_lo = x-dx, x_hi=x+dx, y_lo = y-dy, y_hi = y+dy.
    '''
    return tuple(coord+sgn*delta for sgn in (-1,+1) for coord,delta in zip(pt, (dx,dy)))

In [ ]:
# Verify that the function bounds works as intended
marrakesh_lonlat = (-7.93, 31.67)
dlon, dlat = 0.25, 0.25
marrakesh_rect = bounds(marrakesh_lonlat, dlon, dlat)
print(marrakesh_rect)

+ `geoviews.Rectangles` accepts a list of bounding boxes (each described by a tuple of the form `(x_min, y_min, x_max, y_max)`) to plot.

In [ ]:
rect_opts = opts.Rectangles(
                                line_width=0,
                                alpha=0.25,
                                color='red'
                            )

In [ ]:
rectangle = gv.Rectangles([marrakesh_rect])
(basemap * rectangle).opts( rect_opts )

We'll use the approach above to visualize *Areas of Interest (AOIs)* when constructing search queries for NASA EarthData products. In particular, the convention of representing a bounding box by (left, lower, right, upper) ordinates is also used in the [PySTAC](https://pystac.readthedocs.io/en/stable/) API.

---

## [hvPlot](https://hvplot.holoviz.org/)

+ [hvPlot](https://hvplot.holoviz.org/) is designed to extend the `.plot` API from Pandas DataFrames
+ The code below generates a Pandas DataFrame of simulated oceanographic data. 

- hvplot.pandas.DataFrame.line
- xarray.dataarray.hvplot.quadmesh
- xarray.dataarray.hvplot.image

To include:

+ Example of `pandas.DataFrame.plot.line`
+ Example of `pandas.DataFrame.hvplot.line`
+ Example of `xarray.Dataarray.hvplot.quadmesh`

In [ ]:
import pandas as pd, numpy as np
# Create DataFrame
data = {
    'Time': pd.date_range('2024-01-01', periods=10, freq='M'),
    'Temperature (°C)': np.random.uniform(0, 30, 10),
    'Salinity (ppt)': np.random.uniform(30, 40, 10),
    'Chlorophyll (mg/m³)': np.random.uniform(0, 5, 10)
}
df = pd.DataFrame(data)
df.set_index('Time', inplace=True)

In [ ]:
df.head()

The Pandas DataFrame `.plot` API provides access to a number of plotteing methods. Here, we'll use `.plot.line`, but a range of other options are available (e.g., `.plot.area`, `.plot.bar`, `.plot.hist`, `.plot.scatter`, etc.). This API has been repeated in several libraries due to its convenience.

In [ ]:
# Generates a static Matplotlib plot by default
df.plot. line();

By importing `hvplot.pandas`, a similar interactive plot can be generated. The API for `.hvplot` mimics that for `.plot`. For instance, we can generate the line plot above using `.hvplot.line`. In this case, the default plotting backend is Bokeh, so the plot is *interactive*.

In [ ]:
import hvplot.pandas
import hvplot.xarray
df.hvplot.line()

As usual with Pandas, it is possible to select out portions of the DataFrame as Series; the `.hvplot` API still works.

In [ ]:
df['Temperature (°C)'].hvplot.line()